#

In [2]:
using AlgebraOfGraphics
using AlgebraOfGraphics: density
using CairoMakie
using DataFrames
using DataFramesMeta
using Arrow
using PartialFunctions
using LaTeXStrings

set_aog_theme!()

In [44]:
function load(path::String)
    df = path |> Arrow.Table |> DataFrame |> dropmissing
    
    # if 'radial_distance' is in the dataframe, round it
    if "radial_distance" in names(df)
        df = @chain df begin
            @transform :r = string.(round.(:radial_distance))
        end
    end
    return df
end

load (generic function with 1 method)

In [45]:
"radial_distance" in names(j_events_low_fit)

true

In [48]:


begin
    dir = "../data/05_reporting"

    j_events_low_fit = load("$dir/events.JNO.fit.ts_1.00s_tau_60s.arrow")
    j_events_high_fit = load("$dir/events.JNO.fit.ts_0.12s_tau_60s.arrow")
    j_events_low_der = load("$dir/events.JNO.derivative.ts_1.00s_tau_60s.arrow")
    j_events_high_der = load("$dir/events.JNO.derivative.ts_0.12s_tau_60s.arrow")
    w_events = load("$dir/events.Wind.fit.ts_0.09s_tau_60s.arrow")

    # add a label column to the dataframes
    j_events_low_fit[!, :label] .= "1 Hz (fit)"
    j_events_high_fit[!, :label] .= "8 Hz (fit)"
    j_events_low_der[!, :label] .= "1 Hz (derivative)"
    j_events_high_der[!, :label] .= "8 Hz (derivative)"

    # filter high time resolution events
    j_events_high_fit = @chain j_events_high_fit begin
        filter(:len => >(240), _)
    end

    j_events_der = vcat(j_events_low_der, j_events_high_der)

    # combine the dataframes
    j_events = vcat(j_events_low_fit, j_events_high_fit)

    j_events = @chain j_events begin
        filter(:"fit.stat.rsquared" => >(0.95), _)
    end
end

In [98]:
log_axis = (yscale=log10, xscale=log10)

begin
    di_map = :ion_inertial_length => L"Ion Inertial Length ($km$)"
    di_log_map = :ion_inertial_length => log10 => L"Log Ion Inertial Length ($km$)"
    l_map = :L_k => L"Thickness ($km$)"
    l_log_map = :L_k => log10 => L"Log Thickness ($km$)"
    l_norm_map = :L_k_norm => L"Normalized Thickness ($d_i$)"
    
    jA_map = :j_Alfven => L"Alfvénic Current Density ($nA/m^2$)"
    jA_log_map = :j_Alfven => log10 => L"Log Alfvénic Current Density ($nA/m^2$)"
    j_map = :j0_k => (abs) => L"Current Density ($nA/m^2$)"
    j_log_map = :j0_k => log10 ∘ abs => L"Log Current Density ($nA/m^2$)"
    j_norm_map = :j0_k_norm => L"Normalized Current Density ($J_A$)"

    j_limit = (10^-1, 10^1)
    j_norm_limit = (10^-2, 2)
end


(0.010000000000000002, 2)

In [140]:
begin
    fig = Figure(size=(1000, 800))

    pdflayer = histogram(bins=range(1, 5, length=64), normalization=:pdf)
    datalayer = mapping() * visual(Scatter, markersize=1, color=:white, alpha=0.1)

    layer = pdflayer

    begin
        layer = histogram(bins=range(1, 5, length=64), normalization=:pdf)
        plt = layer * mapping(di_log_map, l_log_map)
        plt1 = data(j_events) * mapping(layout=:r) * plt
        plt2 = data(w_events) * plt
    
        l_log_limit = ((1.5, 3.5), (1.5, 4.5))
        axis = (;limits=l_log_limit)
        draw!(fig[1:2,1:3], plt1, axis=axis)
        draw!(fig[1:2,4:5], plt2, axis=axis) 
    end

    # Current Density Panels
    begin
        layer = histogram(bins=range(-2, 3, length=64), normalization=:pdf)
        plt = layer * mapping(jA_log_map, j_log_map)
        plt3 = data(j_events) * mapping(layout=:r) * plt
        plt4 = data(w_events) * plt
    
        j_log_limit = ((-1, 3), (-2, 2))
        axis = (;limits=j_log_limit)
        draw!(fig[3:4,1:3], plt3, axis=axis)
        draw!(fig[3:4,4:5], plt4, axis=axis)
    end

    # Make ablines across different r panels
    begin
        ablines_data = (; intercepts=[-3,-1,1], slopes=[1,1,1]) 
        lines = data(ablines_data) * mapping(:intercepts, :slopes) * visual(ABLines, linestyle=:dash)
        # draw!(fig[3:4,1:3], lines)
    end

    Label(fig[0,1:3], "Juno in Different Radial Distances", fontsize=20)
    Label(fig[0,4:5], "Wind 11 Hz", fontsize=20)

    fig
end

In [78]:
begin
    fig = Figure(size=(1000, 800))

    pdflayer = density() * visual(Contour)
    # small scatter points
    datalayer = mapping() * visual(Scatter, markersize=3)

    # layer = pdflayer + datalayer
    layer = datalayer
    plt = layer * mapping(di_map, l_map)
    plt1 = data(j_events) * mapping(layout=:r) * plt
    plt2 = data(w_events) * plt

    l_limit = ((10^1, 10^5), (10^1, 10^5))
    axis = merge(log_axis, (;limits=l_limit))
    draw!(fig[1:2,1:3], plt1, axis=axis)
    draw!(fig[1:2,4:5], plt2, axis=axis)

    plt = layer * mapping(jA_map, j_map)
    plt3 = data(j_events) * mapping(layout=:r) * plt
    plt4 = data(w_events) * plt

    j_limit = ((10^-1, 10^3), (10^-2, 10^2))
    axis = merge(log_axis, (;limits=j_limit))
    draw!(fig[3:4,1:3], plt3, axis=axis)
    draw!(fig[3:4,4:5], plt4, axis=axis)


    Label(fig[0,1:3], "Juno in Different Radial Distances", fontsize=20)
    Label(fig[0,4:5], "Wind 11 Hz", fontsize=20)

    fig
end

In [37]:
draw(plt3, axis=(; xscale=log10,limits=j_limit))

In [6]:
begin

    fig = Figure(size=(1000, 500))

    plt = data(j_events) * mapping(col=:r)
    
    draw!( fig[1, 1:5], plt * mapping(:L_k_norm) * histogram(bins=range(0, 50, length=128), normalization=:pdf) )
    draw!( fig[2, 1:5], plt * mapping(:L_k) * histogram(bins=range(0, 3000, length=20), normalization=:pdf) )

    fig
end

In [102]:
function plot_l_r(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:r, color=:label)
    
    grid1 = draw!( fig[1, 1:5], plt * mapping(l_norm_map) * density(datalimits=((0, 50),)) )
    draw!( fig[2, 1:5], plt * mapping(l_map) * density(datalimits=((0, 5000),)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

function plot_j_r(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:r, color=:label)
    
    grid1 = draw!( fig[1, 1:5], plt * mapping(j_map) * density(datalimits=(j_limit,)) )
    draw!( fig[2, 1:5], plt * mapping(j_norm_map) * density(datalimits=(j_norm_limit,)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

function plot_l_j_r(df)
    fig = Figure(size=(1000, 1000))

    plt = data(df) * mapping(col=:r, color=:label)

    grid1 = draw!( fig[1, 1:5], plt * mapping(l_norm_map) * density(datalimits=((0, 50),)) )
    draw!( fig[2, 1:5], plt * mapping(l_map) * density(datalimits=((0, 5000),)) )

    grid1 = draw!( fig[3, 1:5], plt * mapping(j_map) * density(datalimits=(j_limit,)) )
    draw!( fig[4, 1:5], plt * mapping(j_norm_map) * density(datalimits=(j_norm_limit,)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

plot_l_j_r (generic function with 1 method)

In [103]:
plot_l_j_r(j_events_der)

In [89]:
plot_l_r(j_events_der)

In [52]:
plot_l_r(vcat(j_events, j_events_der, cols=:intersect))

In [50]:
plot_l_r(j_events)

In [54]:
function plot_l_r_one(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:label, color=:r)
    
    grid1 = draw!( fig[1, 1:2], plt * mapping(:L_k_norm) * density(datalimits=((0, 50),)) )
    grid2 = draw!( fig[2, 1:2], plt * mapping(:L_k) * density(datalimits=((0, 5000),)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

plot_l_r_one(j_events)

In [55]:
function plot_l_r_one(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:label, color=:r)
    
    grid1 = draw!( fig[1, 1:2], plt * mapping(:L_k_norm) * density(datalimits=((0, 50),)) )
    grid2 = draw!( fig[2, 1:2], plt * mapping(:L_k) * density(datalimits=((0, 5000),)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

plot_l_r_one(j_events_der)

In [66]:
plt = data(j_events) * mapping(:radial_distance, :Alfven_speed, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [64]:
plt = data(j_events) * mapping(:radial_distance, jA_map, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [79]:
using Statistics

In [86]:
# groupby r and describe the data for each group 
# j_events |> @groupby(_.r) |> @map({r=key(_), j0_k=describe(_.j0_k), L_k=describe(_.L_k)})
@chain j_events begin
    groupby(:r)
    combine(:plasma_density =>  mean, :ion_inertial_length => mean, :b_mag => mean) 
end

In [67]:
plt = data(j_events) * mapping(:radial_distance, :plasma_density, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [65]:
plt = data(j_events) * mapping(:radial_distance, di_map, color=:label) * (visual(Scatter) + smooth())
plt |> draw